In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql.functions import lower,col

In [2]:
spark = SparkSession \
    .builder \
    .appName('Db Project') \
    .config("spark.jars", "/spark-measure_2.12-0.17.jar") \
    .getOrCreate()

In [3]:
spark

In [4]:
stagemetrics = StageMetrics(spark)

In [5]:
movies = (spark.read
           .format('csv')
           .option('header', 'true')
           .option('delimiter', ',')
           .option('inferSchema', 'true')
           .load('movie.csv'))

rating = (spark.read
           .format('csv')
           .option('header', 'true')
           .option('delimiter', ',')
           .option('inferSchema', 'true')
           .load('rating.csv'))

tag = (spark.read
           .format('csv')
           .option('header', 'true')
           .option('delimiter', ',')
           .option('inferSchema', 'true')
           .load('tag.csv'))

In [6]:
tag = tag.withColumn('tag', lower(col('tag')))

In [7]:
movies.createOrReplaceTempView('movies')
rating.createOrReplaceTempView('ratings')
tag.createOrReplaceTempView('tags')

In [8]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [9]:
movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [10]:
rating.show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



In [11]:
rating.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: string (nullable = true)



In [12]:
tag.show(5)

+------+-------+-------------+-------------------+
|userId|movieId|          tag|          timestamp|
+------+-------+-------------+-------------------+
|    18|   4141|  mark waters|2009-04-24 18:19:40|
|    65|    208|    dark hero|2013-05-10 01:41:18|
|    65|    353|    dark hero|2013-05-10 01:41:19|
|    65|    521|noir thriller|2013-05-10 01:39:43|
|    65|    592|    dark hero|2013-05-10 01:41:18|
+------+-------+-------------+-------------------+
only showing top 5 rows



In [13]:
tag.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: string (nullable = true)



## Query 1

### Δώστε το πλήθος των χρηστών που είδαν την ταινία “Jumanji”

In [14]:
stagemetrics.begin()
spark.sql("SELECT COUNT(userId) AS users FROM ratings \
           WHERE movieId = (SELECT movieId FROM movies WHERE title='Jumanji (1995)')").show()
stagemetrics.end()

+-----+
|users|
+-----+
|22243|
+-----+



In [ ]:
stagemetrics.print_report()

## Query 2

### Δώστε τα ονόματα των ταινιών που οι χρήστες χαρακτήρισαν ως “boring”

In [16]:
spark.sql("SELECT * FROM movies INNER JOIN tags ON tags.movieId=movies.movieID").show(5)

+-------+--------------------+--------------------+------+-------+-------------+-------------------+
|movieId|               title|              genres|userId|movieId|          tag|          timestamp|
+-------+--------------------+--------------------+------+-------+-------------+-------------------+
|   4141|Head Over Heels (...|      Comedy|Romance|    18|   4141|  mark waters|2009-04-24 18:19:40|
|    208|   Waterworld (1995)|Action|Adventure|...|    65|    208|    dark hero|2013-05-10 01:41:18|
|    353|    Crow, The (1994)|Action|Crime|Fant...|    65|    353|    dark hero|2013-05-10 01:41:19|
|    521|Romeo Is Bleeding...|      Crime|Thriller|    65|    521|noir thriller|2013-05-10 01:39:43|
|    592|       Batman (1989)|Action|Crime|Thri...|    65|    592|    dark hero|2013-05-10 01:41:18|
+-------+--------------------+--------------------+------+-------+-------------+-------------------+
only showing top 5 rows



In [15]:
stagemetrics.begin()
spark.sql("SELECT DISTINCT(title) FROM movies \
           INNER JOIN tags ON tags.movieId=movies.movieID \
           WHERE tag='boring' \
           ORDER BY 1 ASC LIMIT 5").show(truncate=0)
stagemetrics.end()

+------------------------------------+
|title                               |
+------------------------------------+
|(500) Days of Summer (2009)         |
|101 Reykjavik (101 Reykjavík) (2000)|
|12 Years a Slave (2013)             |
|1408 (2007)                         |
|1492: Conquest of Paradise (1992)   |
+------------------------------------+



In [ ]:
stagemetrics.print_report()

## Query 3 

### Δώστε τους χρήστες που έχουν χαρακτηρίσει την ταινία ως “Bollywood” και την έχουν αξιολογήσει με βαθμό >3

In [19]:
spark.sql("SELECT * FROM ratings INNER JOIN tags ON tags.movieId = ratings.movieId AND tags.userId = ratings.userId").show(5)

+------+-------+------+-------------------+------+-------+------------------+-------------------+
|userId|movieId|rating|          timestamp|userId|movieId|               tag|          timestamp|
+------+-------+------+-------------------+------+-------+------------------+-------------------+
| 23171|      1|   3.5|2005-11-02 10:13:17| 23171|      1|             pixar|2006-02-21 11:39:44|
| 88055|      1|   3.0|2012-06-12 20:12:42| 88055|      1|          children|2012-06-12 20:12:52|
| 88055|      1|   3.0|2012-06-12 20:12:42| 88055|      1|computer animation|2012-06-12 20:13:11|
| 88055|      1|   3.0|2012-06-12 20:12:42| 88055|      1|            family|2012-06-12 20:12:55|
| 88055|      1|   3.0|2012-06-12 20:12:42| 88055|      1|          humorous|2012-06-12 20:13:09|
+------+-------+------+-------------------+------+-------+------------------+-------------------+
only showing top 5 rows



In [15]:
stagemetrics.begin()
spark.sql("SELECT DISTINCT(ratings.userId) FROM ratings \
           INNER JOIN tags ON tags.movieId = ratings.movieId AND tags.userId = ratings.userId \
           WHERE tag='bollywood' AND rating>3 \
           ORDER BY ratings.userId LIMIT 5").show()
stagemetrics.end() 

+------+
|userId|
+------+
| 10573|
| 19837|
| 23333|
| 25004|
| 31338|
+------+



In [ ]:
stagemetrics.print_report()

## Query 4

### Βρείτε τις 10 κορυφαίες με βάση το μέσο όρο βαθμολογιών που έχουν λάβει οι ταινίες ανά έτος βαθμολόγησης - 2005

In [21]:
spark.sql("SELECT * FROM movies INNER JOIN ratings ON movies.movieId=ratings.movieId").show(5)

+-------+--------------------+--------------------+------+-------+------+-------------------+
|movieId|               title|              genres|userId|movieId|rating|          timestamp|
+-------+--------------------+--------------------+------+-------+------+-------------------+
|      2|      Jumanji (1995)|Adventure|Childre...|     1|      2|   3.5|2005-04-02 23:53:47|
|     29|City of Lost Chil...|Adventure|Drama|F...|     1|     29|   3.5|2005-04-02 23:31:16|
|     32|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|     1|     32|   3.5|2005-04-02 23:33:39|
|     47|Seven (a.k.a. Se7...|    Mystery|Thriller|     1|     47|   3.5|2005-04-02 23:32:07|
|     50|Usual Suspects, T...|Crime|Mystery|Thr...|     1|     50|   3.5|2005-04-02 23:29:40|
+-------+--------------------+--------------------+------+-------+------+-------------------+
only showing top 5 rows



In [16]:
stagemetrics.begin()
spark.sql("SELECT title,AVG(rating) FROM movies \
           INNER JOIN ratings ON movies.movieId=ratings.movieId \
           WHERE timestamp LIKE '2005%' \
           GROUP BY 1 ORDER BY 2 DESC, movies.title LIMIT 10").show(truncate=0)
stagemetrics.end()

+----------------------------------------------------------------------------+-----------+
|title                                                                       |avg(rating)|
+----------------------------------------------------------------------------+-----------+
|Before the Fall (NaPolA - Elite für den Führer) (2004)                      |5.0        |
|Dancemaker (1998)                                                           |5.0        |
|Fear Strikes Out (1957)                                                     |5.0        |
|Gate of Heavenly Peace, The (1995)                                          |5.0        |
|Life Is Rosy (a.k.a. Life Is Beautiful) (Vie est belle, La) (1987)          |5.0        |
|Married to It (1991)                                                        |5.0        |
|My Life and Times With Antonin Artaud (En compagnie d'Antonin Artaud) (1993)|5.0        |
|Not Love, Just Frenzy (Más que amor, frenesí) (1996)                        |5.0        |

In [ ]:
stagemetrics.print_report()

## Query 5

### Δώστε τις ετικέτες για κάθε ταινία και το όνομα της ταινίας για το έτος 2015 - 5 πρώτες αλφαβητικά

In [22]:
spark.sql("SELECT * FROM movies INNER JOIN tags ON tags.movieId=movies.movieId").show(5)

+-------+--------------------+--------------------+------+-------+-------------+-------------------+
|movieId|               title|              genres|userId|movieId|          tag|          timestamp|
+-------+--------------------+--------------------+------+-------+-------------+-------------------+
|   4141|Head Over Heels (...|      Comedy|Romance|    18|   4141|  mark waters|2009-04-24 18:19:40|
|    208|   Waterworld (1995)|Action|Adventure|...|    65|    208|    dark hero|2013-05-10 01:41:18|
|    353|    Crow, The (1994)|Action|Crime|Fant...|    65|    353|    dark hero|2013-05-10 01:41:19|
|    521|Romeo Is Bleeding...|      Crime|Thriller|    65|    521|noir thriller|2013-05-10 01:39:43|
|    592|       Batman (1989)|Action|Crime|Thri...|    65|    592|    dark hero|2013-05-10 01:41:18|
+-------+--------------------+--------------------+------+-------+-------------+-------------------+
only showing top 5 rows



In [17]:
stagemetrics.begin()
spark.sql("SELECT movies.title as title, collect_set(tags.tag) AS tags FROM movies \
           INNER JOIN tags ON tags.movieId = movies.movieId \
           WHERE tags.timestamp LIKE '2015%' \
           GROUP BY title ORDER BY title LIMIT 5").show()
stagemetrics.end()

+--------------------+--------------------+
|               title|                tags|
+--------------------+--------------------+
|""Great Performan...|              [bd-r]|
|  'burbs, The (1989)|[1980's, dark com...|
|(500) Days of Sum...|[zooey deschanel,...|
|...tick... tick.....|              [bd-r]|
|            1 (2014)|           [sukumar]|
+--------------------+--------------------+



In [ ]:
stagemetrics.print_report()

## Query 6

### Δώστε το πλήθος των ratings για κάθε ταινία - 5 πρώτες με βάση το πλήθος ratings

In [24]:
spark.sql("SELECT * FROM  movies INNER JOIN ratings ON movies.movieId=ratings.movieId").show(5)

+-------+--------------------+--------------------+------+-------+------+-------------------+
|movieId|               title|              genres|userId|movieId|rating|          timestamp|
+-------+--------------------+--------------------+------+-------+------+-------------------+
|      2|      Jumanji (1995)|Adventure|Childre...|     1|      2|   3.5|2005-04-02 23:53:47|
|     29|City of Lost Chil...|Adventure|Drama|F...|     1|     29|   3.5|2005-04-02 23:31:16|
|     32|Twelve Monkeys (a...|Mystery|Sci-Fi|Th...|     1|     32|   3.5|2005-04-02 23:33:39|
|     47|Seven (a.k.a. Se7...|    Mystery|Thriller|     1|     47|   3.5|2005-04-02 23:32:07|
|     50|Usual Suspects, T...|Crime|Mystery|Thr...|     1|     50|   3.5|2005-04-02 23:29:40|
+-------+--------------------+--------------------+------+-------+------+-------------------+
only showing top 5 rows



In [19]:
stagemetrics.begin()
spark.sql("SELECT title,COUNT(rating) FROM movies \
           INNER JOIN ratings ON movies.movieId=ratings.movieId \
           GROUP BY 1 ORDER BY 2 DESC LIMIT 5").show(truncate=0)
stagemetrics.end()

+--------------------------------+-------------+
|title                           |count(rating)|
+--------------------------------+-------------+
|Pulp Fiction (1994)             |67310        |
|Forrest Gump (1994)             |66172        |
|Shawshank Redemption, The (1994)|63366        |
|Silence of the Lambs, The (1991)|63299        |
|Jurassic Park (1993)            |59715        |
+--------------------------------+-------------+



In [ ]:
stagemetrics.print_report()

## Query 7

### Βρείτε τους 10 πρώτους χρήστες με τα περισσότερα rating για κάθε χρονιά - 1995

In [25]:
spark.sql("SELECT * FROM ratings").show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



In [18]:
stagemetrics.begin()
spark.sql("SELECT userId, COUNT(rating) FROM ratings \
           WHERE SUBSTRING(timestamp,1,4) = '1995' \
           GROUP BY userId, SUBSTRING(timestamp,1,4) LIMIT 10").show()
stagemetrics.end()

+------+-------------+
|userId|count(rating)|
+------+-------------+
| 28507|            1|
|131160|            3|
+------+-------------+



In [ ]:
stagemetrics.print_report()

## Query 8

### Βρείτε τις ταινίες με τα περισσότερα ratings για κάθε κατηγορία ταινίας - 5 πρώτες κατηγορίες αλφαβητικά, και την δημοφιλέστερη ταινία σε κάθε μία από αυτές, με βάση το πλήθος ratings

In [39]:
spark.sql("SELECT movies.genres, movies.title, COUNT(ratings.rating) AS count FROM movies \
             INNER JOIN ratings ON ratings.movieId = movies.movieId \
             GROUP BY movies.genres, movies.title ORDER BY genres, count DESC").show(5)

+------------------+--------------------+-----+
|            genres|               title|count|
+------------------+--------------------+-----+
|(no genres listed)|Doctor Who: The T...|   36|
|(no genres listed)|Scorpio Rising (1...|   24|
|(no genres listed)|      At Land (1944)|   10|
|(no genres listed)|  Petting Zoo (2015)|    6|
|(no genres listed)|      Bizarre (2015)|    5|
+------------------+--------------------+-----+
only showing top 5 rows



In [40]:
stagemetrics.begin()
spark.sql("SELECT genres, first(title) as title, MAX(count) as total_ratings FROM \
            (SELECT movies.genres, movies.title, COUNT(ratings.rating) AS count FROM movies \
             INNER JOIN ratings ON ratings.movieId = movies.movieId \
             GROUP BY movies.genres, movies.title ORDER BY genres, count DESC) \
           GROUP BY genres ORDER BY genres LIMIT 5").show(truncate=0)
stagemetrics.end()

+-----------------------------------+------------------------------------------------------------------------------+-------------+
|genres                             |title                                                                         |total_ratings|
+-----------------------------------+------------------------------------------------------------------------------+-------------+
|(no genres listed)                 |Doctor Who: The Time of the Doctor (2013)                                     |36           |
|Action                             |Under Siege 2: Dark Territory (1995)                                          |8335         |
|Action|Adventure                   |Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)|43295        |
|Action|Adventure|Animation         |How to Train Your Dragon 2 (2014)                                             |678          |
|Action|Adventure|Animation|Children|Brave (2012)                                  

In [ ]:
stagemetrics.print_report()

## Query 9

### Δώστε το σύνολο των χρηστών που παρακολουθούν την ίδια ταινία, την ίδια μέρα και ώρα

In [21]:
spark.sql("SELECT movieId, SUBSTRING(timestamp,1,13), COUNT(userId) as total FROM ratings \
             GROUP BY movieId, SUBSTRING(timestamp,1,13) \
             HAVING total > 1 ORDER BY total DESC").show(5)

+-------+---------------------------+-----+
|movieId|substring(timestamp, 1, 13)|total|
+-------+---------------------------+-----+
|   1639|              1998-05-22 15|  271|
|    986|              1998-05-22 13|  194|
|    426|              1998-05-22 11|  134|
|   1639|              1998-05-22 16|  127|
|   1633|              1998-05-22 13|  126|
+-------+---------------------------+-----+
only showing top 5 rows



In [22]:
stagemetrics.begin()
spark.sql("SELECT SUM(total) FROM \
            (SELECT COUNT(userId) as total FROM ratings \
             GROUP BY movieId, SUBSTRING(timestamp,1,13) \
             HAVING total > 1 ORDER BY total DESC)").show()
stagemetrics.end()

+----------+
|sum(total)|
+----------+
|   4280240|
+----------+



In [ ]:
stagemetrics.print_report()

# Query 10

### Δώστε το πλήθος των ταινιών, για κάθε κατηγορία, που οι χρήστες χαρακτήρισαν ως “funny” και με rating > 3.5

In [38]:
spark.sql("SELECT ratings.movieId, movies.genres, ratings.rating, tags.tag FROM ratings \
          INNER JOIN movies ON movies.movieId = ratings.movieId \
          INNER JOIN tags ON tags.movieId = ratings.movieId ").show(5)

+-------+------+------+--------------------+
|movieId|genres|rating|                 tag|
+-------+------+------+--------------------+
|    148| Drama|   3.0|nudity (topless -...|
|    148| Drama|   3.0|    nudity (topless)|
|    148| Drama|   2.0|nudity (topless -...|
|    148| Drama|   2.0|    nudity (topless)|
|    148| Drama|   4.5|nudity (topless -...|
+-------+------+------+--------------------+
only showing top 5 rows



In [23]:
stagemetrics.begin()
spark.sql("SELECT movies.genres, COUNT(ratings.movieId) FROM ratings \
            INNER JOIN movies ON movies.movieId = ratings.movieId \
            INNER JOIN tags ON tags.movieId = ratings.movieId \
            WHERE ratings.rating > 3.5 AND tags.tag LIKE '%funny%' \
            GROUP BY movies.genres ORDER BY genres LIMIT 5").show(truncate=0)
stagemetrics.end()

+--------------------------------------------------+--------------+
|genres                                            |count(movieId)|
+--------------------------------------------------+--------------+
|Action|Adventure                                  |2140          |
|Action|Adventure|Animation|Children|Comedy        |89936         |
|Action|Adventure|Animation|Children|Comedy|Fantasy|1472          |
|Action|Adventure|Animation|Children|Comedy|IMAX   |410           |
|Action|Adventure|Animation|Children|Comedy|Sci-Fi |366           |
+--------------------------------------------------+--------------+



In [ ]:
stagemetrics.print_report()